<a href="https://colab.research.google.com/github/ENRIQUEAHERNANDEZ/TelecomX/blob/main/TelecomX_LATAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#📌 Extracción

In [4]:
# Importamos la biblioteca pandas, que nos ayuda a manejar datos en forma de tablas (como Excel)
import pandas as pd

# Leemos el archivo JSON. Este tipo de archivo tiene estructura en niveles (anidado), como una caja con subcajas.
df_clientes = pd.read_json("/TelecomX_Data.json")

# Mostramos las primeras filas de la tabla para ver cómo están organizados los datos
df_clientes.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


#🔧 Transformación

In [6]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerID  7267 non-null   object
 1   Churn       7267 non-null   object
 2   customer    7267 non-null   object
 3   phone       7267 non-null   object
 4   internet    7267 non-null   object
 5   account     7267 non-null   object
dtypes: object(6)
memory usage: 340.8+ KB


In [8]:
df_clientes.dtypes

,0
customerID,object
Churn,object
customer,object
phone,object
internet,object
account,object


In [9]:
# Ver cuántos valores nulos hay por columna
df_clientes.isnull().sum()

,0
customerID,0
Churn,0
customer,0
phone,0
internet,0
account,0


In [12]:
# 1. Convertimos las columnas anidadas en columnas normales (expandimos en horizontal)
df_aplanado = pd.concat([
    df_clientes[["customerID", "Churn"]],
    df_clientes["customer"].apply(pd.Series),
    df_clientes["phone"].apply(pd.Series),
    df_clientes["internet"].apply(pd.Series),
    df_clientes["account"].apply(pd.Series)
], axis=1)

# 2. Expandimos también los datos de 'Charges' que están dentro de 'account'
df_aplanado = pd.concat([
    df_aplanado.drop(columns=["Charges"]),
    df_aplanado["Charges"].apply(pd.Series).rename(columns={"Monthly": "Charges.Monthly", "Total": "Charges.Total"})
], axis=1)

# 3. Visualizamos las primeras filas para verificar el resultado
df_aplanado.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


In [13]:
df_aplanado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


In [14]:
df_aplanado.dtypes

,0
customerID,object
Churn,object
gender,object
SeniorCitizen,int64
Partner,object
Dependents,object
tenure,int64
PhoneService,object
MultipleLines,object
InternetService,object


In [17]:
# Ver si hay filas duplicadas
df_aplanado.duplicated().sum()

np.int64(0)

In [11]:
# Ejemplo: ver todas las categorías distintas en la columna "Churn"
pd.unique(df_clientes["Churn"])

array(['No', 'Yes', ''], dtype=object)

In [20]:
# 1. Verificar valores nulos por columna
print(" Valores nulos por columna:")
print(df_aplanado.isnull().sum())
print("\n")

# 2. Verificar si hay filas duplicadas
duplicados = df_aplanado.duplicated().sum()
print(f"📎 Filas duplicadas: {duplicados}\n")

# 3. Revisar tipos de datos para detectar errores de formato
print(" Tipos de datos:")
print(df_aplanado.dtypes)
print("\n")

# 4. Revisar valores únicos por columna categórica para detectar inconsistencias
columnas_categoricas = df_aplanado.select_dtypes(include='object').columns

print(" Valores únicos por columna categórica:")
for col in columnas_categoricas:
    print(f"\n {col}:")
    print(df_aplanado[col].unique())

 Valores nulos por columna:
customerID          0
Churn               0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
Charges.Monthly     0
Charges.Total       0
dtype: int64


📎 Filas duplicadas: 0

 Tipos de datos:
customerID           object
Churn                object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contrac

In [22]:
# Copiamos el DataFrame para no modificar el original
df_limpio = df_aplanado.copy()

# 1. Convertimos Charges.Total a numérico (puede estar como texto por errores en el JSON)
df_limpio["Charges.Total"] = pd.to_numeric(df_limpio["Charges.Total"], errors="coerce")

# 2. Eliminamos duplicados (si existieran)
df_limpio.drop_duplicates(inplace=True)

# 3. Corregimos valores vacíos o faltantes en la columna "Churn"
# Por ahora, eliminaremos filas sin valor en "Churn", ya que es la variable objetivo
df_limpio = df_limpio[df_limpio["Churn"].isin(["Yes", "No"])]

# 4. Verificamos y convertimos columnas categóricas a formato uniforme (minúsculas, sin espacios)
columnas_categoricas = df_limpio.select_dtypes(include="object").columns

for col in columnas_categoricas:
    df_limpio[col] = df_limpio[col].str.strip().str.lower()

# 5. Revisamos nuevamente valores nulos
print(" Valores nulos después de limpieza:")
print(df_limpio.isnull().sum())


 Valores nulos después de limpieza:
customerID           0
Churn                0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
Charges.Monthly      0
Charges.Total       11
dtype: int64


/tmp/ipython-input-22-1600960217.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio[col] = df_limpio[col].str.strip().str.lower()


In [23]:
# Creamos la columna 'Cuentas_Diarias' dividiendo el gasto mensual entre 30
df_limpio["cuentas_diarias"] = df_limpio["Charges.Monthly"] / 30

# Mostramos las primeras filas para verificar
df_limpio[["Charges.Monthly", "cuentas_diarias"]].head()

,Charges.Monthly,cuentas_diarias
0,65.6,2.186667
1,59.9,1.996667
2,73.9,2.463333
3,98.0,3.266667
4,83.9,2.796667


In [36]:
df_limpio.rename(columns={
    "churn": "abandono",
    "gender": "genero",
    "seniorcitizen": "adulto_mayor",
    "partner": "tiene_pareja",
    "dependents": "tiene_dependientes",
    "tenure": "meses_contratado",
    "phoneservice": "servicio_telefonico",
    "multiplelines": "lineas_multiples",
    "internetservice": "tipo_internet",
    "onlinesecurity": "seguridad_en_linea",
    "onlinebackup": "respaldo_en_linea",
    "deviceprotection": "proteccion_dispositivo",
    "techsupport": "soporte_tecnico",
    "streamingtv": "tv_en_streaming",
    "streamingmovies": "peliculas_en_streaming",
    "contract": "tipo_contrato",
    "paperlessbilling": "factura_digital",
    "paymentmethod": "forma_pago",
    "Charges.Monthly": "cargos_mensuales",
    "Charges.Total": "cargos_totales",
    "cuentas_diarias": "cuentas_diarias"
}, inplace=True)


In [38]:
print(df_limpio.columns.tolist())

['customerID', 'Churn', 'genero', 'SeniorCitizen', 'Partner', 'Dependents', 'meses_contratado', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'cargos_mensuales', 'cargos_totales', 'cuentas_diarias']


#📊 Carga y análisis

#📄Informe final